# <center>[⏱ Оптимизация памяти и ускорение вычислений](https://stepik.org/lesson/825508/)</center>

### Оглавление ноутбука

<img src='../images/speed_up.png' align="right" width="550" height="550" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">👁 Считывание и сохранение больших датафреймов</a></li>
<li><a href="#c2">🗜 Оптимизация памяти</a></li>
<li><a href="#c3">🥌 Ускорение при помощи `Numpy`</a></li>
<li><a href="#c4">🍢 Векторизация в `pandas`</a></li>
<li><a href="#c5">⚡️ `Numba Jit` - преврати python в ракету</a></li>
<li><a href="#c6">🧵 Multiprocessing - задействуй все ядра</a></li>
<li><a href="#c7">👻 Выводы</a>

</li></ul></font></p>

    

### 🤔 Зачем ускорять и оптимизировать код?

<div class="alert alert-info">
    
Когда речь заходит о работе с действительно большими данными, скорость работы программы и количество памяти, которое ей требуется, могут стать одним из главных ботлнеков в вашей программе.
    
Особенно, если вы ограничены в ресурсах (как например часто бывает на Kaggle) и вам нужно, чтобы ваше решение отработало не только четко, но еще и быстро.

В этом уроке мы рассмотрим подходы и методы, с помощью которых можно сэкономить память и ускорить ваши вычисления.

## Импортируем библиотеки

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# <center> 👁 Считывание и сохранение больших датафреймов </center>

<p id="c1"></p>   

### Скачаем файлы с гугл-диска

<div class="alert alert-info">
Для начала скачаем большой файл с гугл диска, чтобы на нем замерять скорости в дальнейшем

In [2]:
!pip install gdown -q

In [3]:
# для начала скачаем большой файл с гугл диска, чтобы на нем замерять скорости
!gdown 1Sjb6EYfz23ZuqBGYZfkmhWnBbHQBf6Ke -O '../data/blending/'

Downloading...
From (original): https://drive.google.com/uc?id=1Sjb6EYfz23ZuqBGYZfkmhWnBbHQBf6Ke
From (redirected): https://drive.google.com/uc?id=1Sjb6EYfz23ZuqBGYZfkmhWnBbHQBf6Ke&confirm=t&uuid=6b7ab0df-10c0-4685-8c08-134e213b853b
To: /app/storage_local/Course/Competitive_Data_Science/data/blending/text_classification_train.csv
100%|████████████████████████████████████████| 235M/235M [00:09<00:00, 23.9MB/s]


### Работа с `pickle` против `csv`

<div class="alert alert-info">

`Pickle` - это отличная альтернатива привычным нам `.csv` файлам при работе с большими файлами. Мало того, что он считывает и сохраняет всё в разы быстрее, так еще и место на диске такой файл занимает меньше. Также, при использовании `to_pickle()`, сохраняются индексы и все типы колонок, так что при его последующем считывании, датафрейм будет точно таким же и его не нужно будет повторно оптимизировать при каждом открытии, как при использовании `CSV` формата.

In [15]:
%%time

data = pd.read_csv('../data/blending/text_classification_train.csv')

CPU times: user 632 ms, sys: 56.6 ms, total: 688 ms
Wall time: 687 ms


In [16]:
data.shape

(1368, 2623)

In [17]:
%%time

data.to_csv('../data/blending/text_classification_train.csv')

CPU times: user 3.37 s, sys: 92.1 ms, total: 3.46 s
Wall time: 3.82 s


In [18]:
%%time

pd.to_pickle(data, '../data/blending/text_classification_train.pickle')

CPU times: user 5.7 ms, sys: 92.5 ms, total: 98.2 ms
Wall time: 290 ms


In [19]:
%%time

data = pd.read_pickle('../data/blending/text_classification_train.pickle')

CPU times: user 6.5 ms, sys: 19.7 ms, total: 26.2 ms
Wall time: 25.5 ms


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368 entries, 0 to 1367
Columns: 2623 entries, Unnamed: 0.4 to labse_text_feature_767
dtypes: float64(2616), int64(5), object(2)
memory usage: 27.4+ MB


### Считывание по батчам

<div class="alert alert-info">

Если ваш датасет не умещается в память без оптимизации типов или он не нужен вам целиком, то можно считывать по батчам и сразу указывать необходимые типы, индексы и тд. В параметр `chunksize` - передается число сэмплов, которое будет считываться за 1 итерацию.

In [22]:
import gc

chunksize = 1000
tmp_lst = []
with pd.read_csv('../data/car_train.csv',
                 index_col='car_id',
                 dtype={'model': 'category',
                        'car_type': 'category',
                        'fuel_type': 'category',
                        'target_class': 'category'}, chunksize=chunksize) as reader:
    for chunk in reader:
        # chunk = feature(chunk)
        tmp_lst.append(chunk)
        
data = pd.concat(tmp_lst)

del tmp_lst
gc.collect()

data.head()

model  car_type fuel_type  car_rating  year_to_start  \
car_id                                                                         
y13744087j     Kia Rio X-line   economy    petrol        3.78           2015   
O41613818T         VW Polo VI   economy    petrol        3.90           2015   
d-2109686j    Renault Sandero  standart    petrol        6.30           2012   
u29695600e  Mercedes-Benz GLC  business    petrol        4.04           2011   
N-8915870N    Renault Sandero  standart    petrol        4.70           2012   

            riders  year_to_work  target_reg target_class  
car_id                                                     
y13744087j   76163          2021      108.53  another_bug  
O41613818T   78218          2021       35.20  electro_bug  
d-2109686j   23340          2017       38.62   gear_stick  
u29695600e    1263          2020       30.34  engine_fuel  
N-8915870N   26428          2017       30.45  engine_fuel

**Важно:** Пока на какой-то объект в памяти есть ссылки, которые явно не удалили или не переназначили - этот объект будет занимать оперативную память, хотя он может больше не использоваться. Поэтому все временные объекты, которые больше не будут использоваться, лучше явно удалять, используя `del` и после этого запускать сборщик мусора `gc` - `garbage collector`, как в примере выше.
```python
import gc
gc.collect()
```

### Используем генератор

<div class="alert alert-info">
Это особенно актуально, когда мы работаем с картинками - чтобы не хранить их всех в оперативной памяти, они считываются только перед тем, как модель хочет посчитать по ним ошибку и скорректировать веса. Однако, при работе с большими текстами тоже бывает полезно.

In [22]:
def read_file(filename):
    with open(filename, 'r') as f:
        for line in f:
            yield line.strip()

In [23]:
it = read_file('../data/car_info.csv')
next(it)

'car_type,fuel_type,car_rating,year_to_start,riders,car_id,model,target_class,year_to_work,target_reg'

In [24]:
next(it)

'economy,petrol,4.8,2013,42269,P17494612l,Skoda Rapid,engine_overheat,2019,46.65071890960271'

# <center> 🗜 Оптимизация памяти </center>

<p id="c2"></p>   

<div class="alert alert-info">

Самый эффективный способ оптимизации памяти, если вы не хотите удалять часть данных, это установка правильных типов. Если колонка имеет тип `int`, то не нужно ставить ей тип `float`, а если в ней всего несколько уникальных значений, то не нужно делать ее типом `string`. .Например, если в нашем датасете есть бинарная колонка, в которой хранятся только 0 и 1, `pandas` хранит её как максимально возможный тип для целых чисел - `int64`, хотя достаточно будет `int8`. При правильной постановке типов, размер нового датасета обычно в несколько раз меньше (а то и на порядок), чем без них. Также можно вынести какую-то колонку как индекс, чтобы не хранить лишний индекс, но это уже косметика.

### Оптимизируем числовые типы

In [3]:
import pandas as pd

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
df_cars = pd.read_csv('../data/car_train.csv')
df_cars.head()

car_id              model  car_type fuel_type  car_rating  \
0  y13744087j     Kia Rio X-line   economy    petrol        3.78   
1  O41613818T         VW Polo VI   economy    petrol        3.90   
2  d-2109686j    Renault Sandero  standart    petrol        6.30   
3  u29695600e  Mercedes-Benz GLC  business    petrol        4.04   
4  N-8915870N    Renault Sandero  standart    petrol        4.70   

   year_to_start  riders  year_to_work  target_reg target_class  
0           2015   76163          2021      108.53  another_bug  
1           2015   78218          2021       35.20  electro_bug  
2           2012   23340          2017       38.62   gear_stick  
3           2011    1263          2020       30.34  engine_fuel  
4           2012   26428          2017       30.45  engine_fuel

In [40]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_id         2337 non-null   object 
 1   model          2337 non-null   object 
 2   car_type       2337 non-null   object 
 3   fuel_type      2337 non-null   object 
 4   car_rating     2337 non-null   float64
 5   year_to_start  2337 non-null   int64  
 6   riders         2337 non-null   int64  
 7   year_to_work   2337 non-null   int64  
 8   target_reg     2337 non-null   float64
 9   target_class   2337 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 182.7+ KB


In [41]:
df_cars = reduce_mem_usage(df_cars)
df_cars.info()

Memory usage of dataframe is 0.18 MB
Memory usage after optimization is: 0.12 MB
Decreased by 35.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_id         2337 non-null   object 
 1   model          2337 non-null   object 
 2   car_type       2337 non-null   object 
 3   fuel_type      2337 non-null   object 
 4   car_rating     2337 non-null   float16
 5   year_to_start  2337 non-null   int16  
 6   riders         2337 non-null   int32  
 7   year_to_work   2337 non-null   int16  
 8   target_reg     2337 non-null   float16
 9   target_class   2337 non-null   object 
dtypes: float16(2), int16(2), int32(1), object(5)
memory usage: 118.8+ KB


### Оптимизируем категориальные фичи

In [5]:
def convert_columns_to_catg(df, column_list):
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))

In [6]:
%%timeit
df_cars.groupby('model')['car_type'].count()

507 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
convert_columns_to_catg(df_cars, ['model', 'car_type', 'fuel_type', 'target_class'])

converting model                          size:  0.16	->	 0.01
converting car_type                       size:  0.15	->	 0.0
converting fuel_type                      size:  0.15	->	 0.0
converting target_class                   size:  0.16	->	 0.0


In [45]:
%%timeit
df_cars.groupby('model')['car_type'].count()
# При правильной типизации не только уменьшается память, но и возрастает скорость

<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


288 µs ± 4.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [46]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   object  
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float16 
 5   year_to_start  2337 non-null   int16   
 6   riders         2337 non-null   int32   
 7   year_to_work   2337 non-null   int16   
 8   target_reg     2337 non-null   float16 
 9   target_class   2337 non-null   category
dtypes: category(4), float16(2), int16(2), int32(1), object(1)
memory usage: 56.8+ KB


In [47]:
df_cars.memory_usage().sum() / 1024

56.83203125

In [48]:
convert_columns_to_catg(df_cars, ['car_id'])

converting car_id                         size:  0.16	->	 0.23


In [49]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   category
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float16 
 5   year_to_start  2337 non-null   int16   
 6   riders         2337 non-null   int32   
 7   year_to_work   2337 non-null   int16   
 8   target_reg     2337 non-null   float16 
 9   target_class   2337 non-null   category
dtypes: category(5), float16(2), int16(2), int32(1)
memory usage: 125.9 KB


## <center>🥌 Ускорение при помощи `Numpy`</center>

<p id="c3"></p>   

In [2]:
import numpy as np

<div class="alert alert-info">

Первое правило быстрого кода - используйте `numpy` везде, где можно. Он исполняется на чистом `C`, так что работает в сотни раз быстрее обычных циклов и list-ов. В общем, если можете написать код при помощи `numpy` - пишите.

### Инициализация

<div class="alert alert-info">

Демонстрация скорости работы `numpy` на примере инициализации.

In [9]:
%%timeit

a = list(range(1_000_000))

37.7 ms ± 735 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit

b = np.arange(1_000_000)

868 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
a = list(range(1_000_000))

In [5]:
b = np.arange(1_000_000)

In [6]:
%%timeit
100000 in a

563 µs ± 745 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit
100000 in b

528 µs ± 7.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Поэлементные функции

<div class="alert alert-info">

Просто еще раз напомним, что `numpy` сильно быстрее поэлементных функций

In [8]:
%%timeit
[el * el for el in a]

64.4 ms ± 325 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
[el + 10 for el in a]

63 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
b * b

1.08 ms ± 3.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%%timeit
b + 10

975 µs ± 3.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Aггрегирующие функции

<div class="alert alert-info">

Методы и функции `numpy` работают быстрее, чем их обычные аналоги в питоне.

In [12]:
%%timeit

max(b)

70.7 ms ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit

b.max()

379 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Когда использовать `list`?

<div class="alert alert-info">

На самом деле, в одном случае `list` все-таки может быть быстрее. Если вы хотите добавлять много новых элементов, то добавление одного элемента в `list` происходит за O(1) и суммарно на всё тратится O(n) времени, а при добавлении в `numpy` массив, нам нужно его заново весь проинициализировать, даже если добавили всего один элемент, так что суммарное время работы будет O(n^2).

In [14]:
%%timeit

a = np.zeros(0)
for el in range(1000):
    b = np.zeros(1000)
    a = np.append(a, b)

317 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit

a = []
for el in range(1000):
    b = [0 for i in range(1000)]
    a += b

31.8 ms ± 408 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Используем `set()`

<div class="alert alert-info">

Если нужно проверить наличие элементов в другом массиве и тот "другой" достаточно большой, то лучше использовать `set()`, так как в нем поиск элемента осуществляется за `O(log(n))`, а в `np.isin()` за `O(n)`. Так что, даже несмотря на то, что `numpy` оптимизирован, `set()` выигрывает его по времени.

In [16]:
a = np.arange(1000)
b = np.arange(1000000) * -1

In [17]:
%%timeit
np.isin(a, b)

5.45 ms ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
st = set(b)

In [19]:
%%timeit
[el in st for el in a]

91.4 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## <center> 🍢 Векторизация в `pandas` </center>

<p id="c4"></p>   

<div class="alert alert-info">
Векторизация - это процесс, когда мы заменяем обычные поэлементные функции на операции сразу со всем вектором значений, что позволяет значительно прибавить в скорости.

In [26]:
df_cars.head()

car_id              model  car_type fuel_type  car_rating  \
0  y13744087j     Kia Rio X-line   economy    petrol        3.78   
1  O41613818T         VW Polo VI   economy    petrol        3.90   
2  d-2109686j    Renault Sandero  standart    petrol        6.30   
3  u29695600e  Mercedes-Benz GLC  business    petrol        4.04   
4  N-8915870N    Renault Sandero  standart    petrol        4.70   

   year_to_start  riders  year_to_work  target_reg target_class  
0           2015   76163          2021      108.53  another_bug  
1           2015   78218          2021       35.20  electro_bug  
2           2012   23340          2017       38.62   gear_stick  
3           2011    1263          2020       30.34  engine_fuel  
4           2012   26428          2017       30.45  engine_fuel

In [28]:
df_cars['car_type'].unique()

array(['economy', 'standart', 'business', 'premium'], dtype=object)

### `Numpy.where()`

<div class="alert alert-info">

Позволяет проверить какое-то условие и в зависимости от его результатов вернуть, то или иное значение. Векторный аналог `if-else`.

In [29]:
def simple_if(x):
    if x['car_rating'] < 3.78:
        return x['car_type']
    else:
        return x['fuel_type']

In [30]:
%%timeit
df_cars.apply(simple_if, axis=1)

21.9 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%timeit
# первое значение - это условие, 
# второе - что возвращать, если выполнено условие 
# третье - что возвращать, если не выполнено условие
np.where(df_cars['car_rating'].values < 3.78, df_cars['car_type'].values, df_cars['fuel_type'].values)

50.1 µs ± 702 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### `Numpy.vectorize()`

<div class="alert alert-info">
Часть функции можно оставить поэлементными и просто засчет их векторизации, получить значительный прирост скорости. Однако работает это все равно медленнее, чем операции с векторами чисел.

In [39]:
def simple_if2(car_rating, car_type, fuel_type):
    if car_rating < 3.78:
        return car_type
    else:
        return fuel_type

In [40]:
vectfunc = np.vectorize(simple_if2)

In [41]:
%%timeit
vectfunc(df_cars['car_rating'], df_cars['car_type'], df_cars['fuel_type'])

726 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### `Numpy.select()`

<div class="alert alert-info">

Как `np.where()`, только для нескольких условий.

In [42]:
import re

def hard_if(x):
    if x['car_rating'] < 3:
        if 'Audi' == x['model']:
            return 0
        else:
            return 1
    elif x['car_rating'] in [3, 4, 5]:
        return 2
    else:
        return 3

In [43]:
%%timeit
df_cars.apply(simple_if, axis=1)

21.8 ms ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit
conditions = [
    (df_cars['car_rating'] < 3) & (df_cars['model'] == 'Audi'),
    (df_cars['car_rating'] < 3),
    df_cars['car_rating'].isin([3, 4, 5])
]

choices = [0, 1, 2]
np.select(conditions, choices, default=3)

835 µs ± 7.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Переписываем словари

<div class="alert alert-info">
На самом деле, обращаться к значению из словаря тоже можно быстро.

In [45]:
mydict = {'economy': 0,
          'standart': 1,
          'business': 2,
          'premium': 3}

def f(x):
    if x['car_rating'] > 5:
        return mydict[x['car_type']]
    else:
        return np.nan

In [51]:
%%timeit
df_cars.apply(f, axis=1)

17.6 ms ± 79.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%%timeit
np.where(df_cars['car_rating'] > 5, df_cars['car_type'].map(mydict), np.nan)

518 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Пишем `groupby` на `numpy`

<div class="alert alert-info">

`pd.groupby()` - одна из самых часто используемых и полезных функций в пандасе, которую можно ускорить до 30 раз при помощи `numpy`!

In [20]:
import pandas as pd

df_cars = pd.read_csv('../data/car_train.csv')
df_cars.head()
# df_cars.info()

car_id              model  car_type fuel_type  car_rating  \
0  y13744087j     Kia Rio X-line   economy    petrol        3.78   
1  O41613818T         VW Polo VI   economy    petrol        3.90   
2  d-2109686j    Renault Sandero  standart    petrol        6.30   
3  u29695600e  Mercedes-Benz GLC  business    petrol        4.04   
4  N-8915870N    Renault Sandero  standart    petrol        4.70   

   year_to_start  riders  year_to_work  target_reg target_class  
0           2015   76163          2021      108.53  another_bug  
1           2015   78218          2021       35.20  electro_bug  
2           2012   23340          2017       38.62   gear_stick  
3           2011    1263          2020       30.34  engine_fuel  
4           2012   26428          2017       30.45  engine_fuel

### Считаем количество значений в группе

<div class="alert alert-info">

Для того, чтобы посчитать кол-во значений в каждой группе, мы можем воспользоваться функцией `np.bincount()`

In [6]:
%%timeit
df_cars.groupby(['model', 'fuel_type'])['target_reg'].count()

813 µs ± 3.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
df_cars['int_model'] = lbl.fit_transform((df_cars['model'] + df_cars['fuel_type']).astype(str))

In [8]:
%%timeit
np.bincount(df_cars['int_model'])

12.6 µs ± 62.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
gb_values = df_cars.groupby(['int_model'])['target_reg'].count()
np_values = np.bincount(df_cars['int_model'])

(gb_values == np_values).all()

True

Пример работы `np.bincount`:

<center> <img src = '../images/numpy-bincount.png' width=550>
<!-- ![image.png](attachment:4034fb01-3672-485b-b44c-e57537aa033b.png) -->

### Считаем сумму/среднее

<div class="alert alert-info">

Для того, чтобы посчитать сумму значений в каждой группе, мы можем воспользоваться `np.bincount(weights=your_weights)`

In [10]:
%%timeit
df_cars.groupby(['model', 'fuel_type'])['target_reg'].sum()

869 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%%timeit
np.bincount(df_cars['int_model'], weights=df_cars['target_reg'])

21.4 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
gb_values = df_cars.groupby(['int_model'])['target_reg'].sum()
np_values = np.bincount(df_cars['int_model'], weights=df_cars['target_reg'])

(gb_values.round(10) == np_values.round(10)).all()

True

### Считаем минимум/максимум

<div class="alert alert-info">

Для того, чтобы посчитать такие функцие как максимум, минимум, произведение и т.д., нам потребуется `np.ufunc.reduceat()`

In [13]:
%%timeit
df_cars.groupby(['int_model'])['target_reg'].agg(['max'])

496 µs ± 4.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit
indices = df_cars['int_model']
max_values = np.maximum.reduceat(df_cars['target_reg'].values[np.argsort(indices)],
                                 np.concatenate(([0], np.cumsum(np.bincount(indices))))[:-1])
max_values

189 µs ± 495 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
indices = df_cars['int_model']
np_values = np.maximum.reduceat(df_cars['target_reg'].values[np.argsort(indices)],
                                 np.concatenate(([0], np.cumsum(np.bincount(indices))))[:-1])

gb_values = df_cars.groupby(['int_model'])['target_reg'].agg('max')
(gb_values.round(10) == np_values.round(10)).all()

True

In [17]:
# возвращает перестановку чисел от 0 до n - 1, в которой элементы отсортированы по возрастанию
np.argsort(indices).values, df_cars['int_model'].values[np.argsort(indices)]

(array([1168,  645,  189, ..., 1209, 1123, 1659]),
 array([ 0,  0,  0, ..., 25, 25, 25]))

In [18]:
# i-ый элемент хранит в себе сумму элементов первых i значений
np.bincount(indices), np.cumsum(np.bincount(indices))

(array([ 17,  20,  21,  16,  18, 161, 140, 143, 111, 147,  21,  23,  17,
         22, 146, 154, 152, 147, 118, 158, 130,  14, 152, 141, 135,  13]),
 array([  17,   37,   58,   74,   92,  253,  393,  536,  647,  794,  815,
         838,  855,  877, 1023, 1177, 1329, 1476, 1594, 1752, 1882, 1896,
        2048, 2189, 2324, 2337]))

In [19]:
# сдвигаем так, чтобы значения массива означали позиции начала групп
np.concatenate(([0], np.cumsum(np.bincount(indices))))[:-1]

array([   0,   17,   37,   58,   74,   92,  253,  393,  536,  647,  794,
        815,  838,  855,  877, 1023, 1177, 1329, 1476, 1594, 1752, 1882,
       1896, 2048, 2189, 2324])

## <center> ⚡️ `Numba Jit` - преврати python в ракету </center>

<p id="c5"></p>   

<div class="alert alert-info">

Если нужно что-то кастомное, что нельзя переписать на `numpy`, но нужно чтобы работало быстро, то `numba.jit` вам в помощь. Он конвертирует, написанный вами на питоне, код в `С` и засчет этого работает на порядок (а иногда и на несколько) быстрее, чем обычный питон. Однако, поддерживает он не весь питоновский функционал и внутри него нельзя использовать разные библиотечные функции, так что в основном используется для низкоуровневой оптимизации.

In [21]:
!pip install numba -q

In [25]:
from numba import jit

@jit(nopython=True)
def f(n):
    s = 0.
    for i in range(n):
        s += i ** 0.5
    return s

In [24]:
%%timeit
f(10000)

1.31 ms ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%%timeit
f(10000)

26.9 µs ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
@jit(nopython=True)
def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value
    return a

In [29]:
%%timeit
monotonically_increasing(df_cars['target_reg'].values)

551 µs ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [30]:
%%timeit
monotonically_increasing(df_cars['target_reg'].values)

530 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## <center> 🧵 Multiprocessing </center>
<p id="c6"></p>   

<div class="alert alert-info">
 
С помощью `multiprocessing`, можно ускорить вообще практически все. Он позвлоляет использовать не одно ядро вашего компьютера, а сразу несколько и, соответственно, ускорить вычисления (уже не только `io-bound`, но еще и `cpu-bound`) в количество раз, пропорциональное количеству доступных ядер.

In [33]:
!pip install pymorphy2 -q

In [ ]:
df = pd.read_pickle('../data/blending/text_transformer_data.pkl')
df.head()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def text_prepare(text):
    lemmatizer = pymorphy2.MorphAnalyzer()
    word_tokens = word_tokenize(text)
    word_tokens = [w for w in word_tokens if not w in stop_words]
    word_tokens = [lemmatizer.parse(w)[0].normal_form for w in word_tokens]
    filtered_text = ' '.join(word_tokens)
    return filtered_text

In [ ]:
%%time

tmp = df['text'].apply(text_prepare)

In [ ]:
from multiprocessing import Pool

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    
    df_split = np.array_split(df, n_cores)
    
    pool = Pool(n_cores)
    
    df = np.concatenate(pool.map(func, df_split))
    
    pool.close()
    pool.join()
    return df

def many_row_prepare(df, text_col='text'):
    res = []
    for text in df[text_col]:
        res.append(text_prepare(text))
    return res

In [ ]:
%%time
tmp = parallelize_dataframe(df, many_row_prepare)

## <center> 👻 Выводы </center>

<p id="c7"></p>   

<div class="alert alert-info">
    
Ускорение вычислений и оптимизация памяти - это важные задачи, с которыми периодически можно столкнуться во время написания соревнований или просто при работе с большим количеством данных. В данном уроке мы рассмотрели основные способы решения этих задач и как их применять на практике.
    
Если тебе нужно ускорить твой код, то:
* Замени всё, что можно, на `numpy`
* Оставшееся перепиши на `numba.jit`
* Если предыдущих двух пунктов не хватило, то используй `multiprocessing`

Если нужно оптимизировать память, то:
* Примени хранение/считывание при помощи `pickle`
* Примени правильное расставление типов
* Если тебе не нужен весь датасет за раз, то можно считывать его по частям